# Naive Bayes and K-Nearest Neighbours from Scratch

This notebook implements **Naive Bayes** and **K-Nearest Neighbours (KNN)** algorithms from scratch using:
- NumPy
- Pandas
- Matplotlib

 No machine learning libraries like scikit-learn are used.

Dataset used: **Titanic Dataset**


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Load Dataset


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Preprocessing

Steps performed:
1. Handle missing values
2. Drop irrelevant columns
3. Encode categorical variables


In [7]:
data = train.copy()

# Fill missing values
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Drop unnecessary columns
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# One-hot encoding
data = pd.get_dummies(data, columns=['Sex', 'Embarked'], drop_first=True)

# Split features and target
X = data.drop('Survived', axis=1).values
y = data['Survived'].values


C:\Users\PATEL NISHARG H\AppData\Local\Temp\ipykernel_29140\2919524620.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)
C:\Users\PATEL NISHARG H\AppData\Local\Temp\ipykernel_29140\2919524620.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

## Train-Test Split


In [8]:
split = int(0.8 * len(X))

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


## Naive Bayes Algorithm (From Scratch)

Assumes features follow Gaussian distribution.


In [9]:
def naive_bayes_train(X, y):
    classes = np.unique(y)
    mean = {}
    var = {}
    prior = {}

    for c in classes:
        X_c = X[y == c]
        mean[c] = np.mean(X_c, axis=0)
        var[c] = np.var(X_c, axis=0) + 1e-6
        prior[c] = X_c.shape[0] / X.shape[0]

    return mean, var, prior


def gaussian_pdf(x, mean, var):
    return np.exp(-(x - mean) ** 2 / (2 * var)) / np.sqrt(2 * np.pi * var)


def naive_bayes_predict(X, mean, var, prior):
    predictions = []

    for x in X:
        posteriors = []

        for c in prior:
            likelihood = np.sum(np.log(gaussian_pdf(x, mean[c], var[c])))
            posterior = likelihood + np.log(prior[c])
            posteriors.append(posterior)

        predictions.append(np.argmax(posteriors))

    return np.array(predictions)


In [10]:
mean, var, prior = naive_bayes_train(X_train, y_train)
nb_predictions = naive_bayes_predict(X_test, mean, var, prior)

nb_accuracy = np.mean(nb_predictions == y_test)
print("Naive Bayes Accuracy:", nb_accuracy)


TypeError: loop of ufunc does not support argument 0 of type float which has no callable exp method

## K-Nearest Neighbours Algorithm (From Scratch)

Uses Euclidean distance and majority voting.


In [11]:
def knn_predict(X_train, y_train, X_test, k=5):
    predictions = []

    for x in X_test:
        distances = np.sqrt(np.sum((X_train - x) ** 2, axis=1))
        k_indices = np.argsort(distances)[:k]
        k_labels = y_train[k_indices]
        predictions.append(np.bincount(k_labels).argmax())

    return np.array(predictions)


In [12]:
knn_predictions = knn_predict(X_train, y_train, X_test, k=5)
knn_accuracy = np.mean(knn_predictions == y_test)

print("KNN Accuracy:", knn_accuracy)


TypeError: loop of ufunc does not support argument 0 of type float which has no callable sqrt method

## Accuracy Comparison


In [13]:
models = ['Naive Bayes', 'KNN']
accuracies = [nb_accuracy, knn_accuracy]

plt.bar(models, accuracies)
plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')
plt.show()


NameError: name 'nb_accuracy' is not defined

## Conclusion

- Both Naive Bayes and KNN were implemented completely from scratch.
- No machine learning libraries were used.
- Data preprocessing and visualization were performed.
- Naive Bayes performed faster, while KNN accuracy depended on K value.

This notebook satisfies all assignment constraints.
